In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import boto3
import io

In [2]:
# Sagemaker doesn't have these libraries 

from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier

### Load data from S3

In [3]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='features_events.csv')
events = pd.read_csv(io.BytesIO(obj['Body'].read())).set_index('user_id')
events.head()

,purchase_count_total,purchase_count_last_week,purchase_count_2_weeks,purchase_sum_total,purchase_sum_last_week,purchase_sum_2_weeks
user_id,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='features_events.csv')
events = pd.read_csv(io.BytesIO(obj['Body'].read()))
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='features_events_full.csv')
events_full = pd.read_csv(io.BytesIO(obj['Body'].read())).set_index('user_id')
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='features_session_training6.0.csv')
sessions = pd.read_csv(io.BytesIO(obj['Body'].read()))
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='features_session_full6.0.csv')
sessions_full = pd.read_csv(io.BytesIO(obj['Body'].read()))
sessions.head()

In [9]:
attributes = pd.read_csv('features_attr2.csv').set_index('user_id')
attributes_full = pd.read_csv('features_attr2_full.csv').set_index('user_id')
attributes.head()

,attr_0,attr_1,attr_2,attr_3,attr_5,attr_7,attr_8,attr_13,attr_14,attr_15,attr_25,attr_27
user_id,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
sessions_full['user_id'] = sessions_full['id_']
sessions_full = sessions_full.drop('id_', axis=1).set_index('user_id')
sessions['user_id'] = sessions['id_']
sessions = sessions.drop('id_', axis=1).set_index('user_id')
sessions.head()

,cr_from_now,is_developer3,online_ss_pct,last2week_ss_duration,total_count,last2week_count,last5day_count,weekend_count,weekday_count,r1,...,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20
user_id,,,,,,,,,,,,,,,,,,,,,
0,38.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,31.0,1.0,0.0,5980318.0,9.0,1.0,0.0,1.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,27.0,1.0,0.0,0.0,8.0,0.0,0.0,7.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.0,1.0,0.0,1034377.0,2.0,2.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.0,1.0,0.0,1361068.0,3.0,3.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='labels14.csv')
labels14 = pd.read_csv(io.BytesIO(obj['Body'].read())).set_index('user_id')
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='labels7.csv')
labels7 = pd.read_csv(io.BytesIO(obj['Body'].read())).set_index('user_id')

In [13]:
obj = s3.get_object(Bucket='ec-ml2-final-project', Key='user_dict.csv')
users = pd.read_csv(io.BytesIO(obj['Body'].read()), header=None)
inv_user_dict = {row[1][1]:row[1][0] for row in users.iterrows()}

In [14]:
events2 = pd.read_csv('features_events2.csv').set_index('user_id')
events2_full = pd.read_csv('features_events2_full.csv').set_index('user_id')
events2_full.head()

,e3_count_1_week,e4_count_1_week,e5_count_1_week,e6_count_1_week,e7_count_1_week,e40_count_1_week,e41_count_1_week,e42_count_1_week,e3_count_2_week,e4_count_2_week,e5_count_2_week,e6_count_2_week,e7_count_2_week,e40_count_2_week,e41_count_2_week,e42_count_2_week
user_id,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Explore attribute correlation with target

In [18]:
att1 = attributes.iloc[:,:37]
att2 = attributes.iloc[:,37:]
d1_14 = labels14.join(att1)
d2_14 = labels14.join(att2)
d1_7 = labels7.join(att1)
d2_7 = labels7.join(att2)

In [19]:
corr = d1_14.corr()
corr.style.background_gradient()

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/matplotlib/colors.py:504: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,label,episode_2,episode,episode_3,attr_28,attr_24,attr_29,attr_0,attr_1,attr_2,attr_3,attr_5,attr_7,attr_8,attr_10,attr_13,attr_14,attr_15,attr_16,attr_17,attr_18,attr_19,attr_19.1,attr_24.1,attr_25,attr_26,attr_27
label,1,0.0799405,0.0993729,0.00274733,0.00195612,-0.00506222,-0.0423963,0.142845,0.223502,0.267584,0.161631,0.167106,0.106211,0.169827,-0.00922243,0.374057,0.397882,0.0556094,-0.0706934,-0.0157801,0.0763748,-0.0619511,-0.0619511,-0.00506222,0.170524,-0.000392363,0.170523
episode_2,0.0799405,1,0.835062,-0.0129388,-0.000163619,0.0675426,-0.20109,0.212718,0.151653,0.167824,0.241363,0.234135,0.197702,0.212779,0.131979,0.210973,0.18551,0.0804913,-0.32854,0.182478,0.22007,-0.475137,-0.475137,0.0675426,0.244102,-6.76483e-05,0.244103
episode,0.0993729,0.835062,1,-0.0120386,0.00474541,0.0805578,-0.241496,0.262201,0.200311,0.205082,0.296855,0.283499,0.256666,0.257669,0.145518,0.251832,0.236382,0.090981,-0.476858,0.233774,0.290985,-0.590206,-0.590206,0.0805578,0.302379,0.00110453,0.302378
episode_3,0.00274733,-0.0129388,-0.0120386,1,0.00487121,-0.0037042,nan,-0.0820638,-0.0111205,-0.00772381,-0.0940678,-0.0932769,-0.085743,-0.0785332,-0.133546,-0.0257215,-0.0360022,-0.033995,0.0306869,-0.0282502,-0.0447788,0.0244097,0.0244097,-0.0037042,-0.0285569,-0.00428685,-0.0285529
attr_28,0.00195612,-0.000163619,0.00474541,0.00487121,1,nan,nan,-0.0176984,-0.0184932,0.000260277,-0.00231625,-0.012093,-0.016346,-0.00959421,0.000313922,-0.00965029,-0.025962,-0.0011995,-0.0164748,0.00261641,0.00319213,-0.00621196,-0.00621196,nan,-0.00405272,-0.0128323,-0.0040405
attr_24,-0.00506222,0.0675426,0.0805578,-0.0037042,nan,1,0.140238,-0.073342,-0.0259162,-0.0180705,-0.0264766,-0.0575939,-0.0132118,-0.0573908,-0.00125842,-0.11529,-0.0735563,-0.0360851,-0.0801626,0.0398974,-0.119565,-0.124039,-0.124039,1,-0.045958,nan,-0.045958
attr_29,-0.0423963,-0.20109,-0.241496,nan,nan,0.140238,1,-0.155828,-0.059568,-0.0258394,-0.197543,-0.239342,-0.215718,-0.238066,-0.0145211,-0.0902678,-0.245249,-0.139069,0.335391,-0.258595,-0.321973,0.361957,0.361957,0.140238,-0.209699,nan,-0.209699
attr_0,0.142845,0.212718,0.262201,-0.0820638,-0.0176984,-0.073342,-0.155828,1,0.155948,0.121251,0.858445,0.91862,0.726426,0.968891,0.718967,0.494886,0.568476,0.54925,-0.161665,0.367869,0.183086,-0.163314,-0.163314,-0.073342,0.71097,0.00988466,0.711844
attr_1,0.223502,0.151653,0.200311,-0.0111205,-0.0184932,-0.0259162,-0.059568,0.155948,1,0.62877,0.233841,0.23031,0.121544,0.215308,0.0317443,0.168287,0.419351,0.118555,-0.0802185,-0.00554553,0.065848,-0.0741022,-0.0741022,-0.0259162,0.286273,0.0176689,0.286269
attr_2,0.267584,0.167824,0.205082,-0.00772381,0.000260277,-0.0180705,-0.0258394,0.121251,0.62877,1,0.151026,0.182515,0.0957781,0.17669,0.0101238,0.17389,0.399975,0.0970624,-0.0545676,0.036811,-0.0149774,-0.0536382,-0.0536382,-0.0180705,0.210784,0.0145833,0.210779


In [20]:
corr = d2_14.corr()
corr.style.background_gradient()

,label,attr_19.2,attr_19.3,attr_19.4,attr_19.5,attr_19.1.1,attr_19.2.1,attr_19.3.1,attr_19.4.1,attr_19.5.1,attr_19.6,attr_19.7,attr_19.8,attr_19.9,attr_24.2,attr_24.3,attr_25.1,attr_26.1,attr_27.1,attr_19.10,attr_19.11,attr_19.12,attr_19.13,attr_19.14,attr_19.15,attr_19.16,attr_19.17,attr_19.18,attr_19.19,attr_19.20,attr_19.21,attr_19.22,attr_19.23,attr_19.24,vad_pos,vad_neg,vad_neu,vad_compound,ave_lenword
label,1,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.00506222,-0.00506222,0.170524,-0.000392363,0.170523,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0619511,-0.0258434,-0.0074122,-0.0248987,-0.0215981,-0.0597205
attr_19.2,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.3,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.4,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.5,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.1.1,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.2.1,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.3.1,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372
attr_19.4.1,-0.0619511,1,1,1,1,1,1,1,1,1,1,1,1,1,-0.124039,-0.124039,-0.193413,-0.00395698,-0.193412,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.0512759,0.0052793,0.0535326,0.0409222,0.129372


In [22]:
corr = d1_7.corr()
corr.style.background_gradient()

,label,episode_2,episode,episode_3,attr_28,attr_24,attr_29,attr_0,attr_1,attr_2,attr_3,attr_5,attr_7,attr_8,attr_10,attr_13,attr_14,attr_15,attr_16,attr_17,attr_18,attr_19,attr_19.1,attr_24.1,attr_25,attr_26,attr_27
label,1,0.0705211,0.0866765,0.00264059,0.00164612,-0.00367237,-0.0285799,0.133708,0.200567,0.241411,0.137194,0.151969,0.0917735,0.157345,-0.00757278,0.35788,0.3827,0.0598999,-0.0599706,-0.0141117,0.0653429,-0.0528883,-0.0528883,-0.00367237,0.146731,-0.000330313,0.146731
episode_2,0.0705211,1,0.835062,-0.0129388,-0.000163619,0.0675426,-0.20109,0.212718,0.151653,0.167824,0.241363,0.234135,0.197702,0.212779,0.131979,0.210973,0.18551,0.0804913,-0.32854,0.182478,0.22007,-0.475137,-0.475137,0.0675426,0.244102,-6.76483e-05,0.244103
episode,0.0866765,0.835062,1,-0.0120386,0.00474541,0.0805578,-0.241496,0.262201,0.200311,0.205082,0.296855,0.283499,0.256666,0.257669,0.145518,0.251832,0.236382,0.090981,-0.476858,0.233774,0.290985,-0.590206,-0.590206,0.0805578,0.302379,0.00110453,0.302378
episode_3,0.00264059,-0.0129388,-0.0120386,1,0.00487121,-0.0037042,nan,-0.0820638,-0.0111205,-0.00772381,-0.0940678,-0.0932769,-0.085743,-0.0785332,-0.133546,-0.0257215,-0.0360022,-0.033995,0.0306869,-0.0282502,-0.0447788,0.0244097,0.0244097,-0.0037042,-0.0285569,-0.00428685,-0.0285529
attr_28,0.00164612,-0.000163619,0.00474541,0.00487121,1,nan,nan,-0.0176984,-0.0184932,0.000260277,-0.00231625,-0.012093,-0.016346,-0.00959421,0.000313922,-0.00965029,-0.025962,-0.0011995,-0.0164748,0.00261641,0.00319213,-0.00621196,-0.00621196,nan,-0.00405272,-0.0128323,-0.0040405
attr_24,-0.00367237,0.0675426,0.0805578,-0.0037042,nan,1,0.140238,-0.073342,-0.0259162,-0.0180705,-0.0264766,-0.0575939,-0.0132118,-0.0573908,-0.00125842,-0.11529,-0.0735563,-0.0360851,-0.0801626,0.0398974,-0.119565,-0.124039,-0.124039,1,-0.045958,nan,-0.045958
attr_29,-0.0285799,-0.20109,-0.241496,nan,nan,0.140238,1,-0.155828,-0.059568,-0.0258394,-0.197543,-0.239342,-0.215718,-0.238066,-0.0145211,-0.0902678,-0.245249,-0.139069,0.335391,-0.258595,-0.321973,0.361957,0.361957,0.140238,-0.209699,nan,-0.209699
attr_0,0.133708,0.212718,0.262201,-0.0820638,-0.0176984,-0.073342,-0.155828,1,0.155948,0.121251,0.858445,0.91862,0.726426,0.968891,0.718967,0.494886,0.568476,0.54925,-0.161665,0.367869,0.183086,-0.163314,-0.163314,-0.073342,0.71097,0.00988466,0.711844
attr_1,0.200567,0.151653,0.200311,-0.0111205,-0.0184932,-0.0259162,-0.059568,0.155948,1,0.62877,0.233841,0.23031,0.121544,0.215308,0.0317443,0.168287,0.419351,0.118555,-0.0802185,-0.00554553,0.065848,-0.0741022,-0.0741022,-0.0259162,0.286273,0.0176689,0.286269
attr_2,0.241411,0.167824,0.205082,-0.00772381,0.000260277,-0.0180705,-0.0258394,0.121251,0.62877,1,0.151026,0.182515,0.0957781,0.17669,0.0101238,0.17389,0.399975,0.0970624,-0.0545676,0.036811,-0.0149774,-0.0536382,-0.0536382,-0.0180705,0.210784,0.0145833,0.210779


In [10]:
attr0 = attributes['attr_0'].dropna().groupby('user_id').agg('mean')
attr1 = attributes['attr_1'].dropna().groupby('user_id').agg('mean')
attr2 = attributes['attr_2'].dropna().groupby('user_id').agg('mean')
attr3 = attributes['attr_3'].dropna().groupby('user_id').agg('mean')
attr5 = attributes['attr_5'].dropna().groupby('user_id').agg('mean')
attr7 = attributes['attr_7'].dropna().groupby('user_id').agg('mean')
attr8 = attributes['attr_8'].dropna().groupby('user_id').agg('mean')
attr13 = attributes['attr_13'].dropna().groupby('user_id').agg('mean')
attr14 = attributes['attr_14'].dropna().groupby('user_id').agg('mean')
attr25 = attributes['attr_25'].dropna().groupby('user_id').agg('mean')
attr251 = attributes['attr_25.1'].dropna().groupby('user_id').agg('mean')
attr27 = attributes['attr_27'].dropna().groupby('user_id').agg('mean')
attr271 = attributes['attr_27.1'].dropna().groupby('user_id').agg('mean')

In [83]:
X_full = events_full.join(events2_full).join(sessions_full)
data14 = labels14.join(events).join(events2).join(sessions)
data7 = labels7.join(events).join(events2).join(sessions)
attrs = [attr0, attr1, attr2, attr3, attr5, attr7, attr8, attr13, attr14, attr25, attr251, attr27, attr271]
for attr in attrs:
    X_full = X_full.join(attr)
    data14 = data14.join(attr)
    data7 = data7.join(attr)

In [92]:
# create features using best attributes
features_attr = X_full[['attr_13', 'attr_14', 'attr_1','attr_8','attr_5','attr_0']]
features_attr.head()

,attr_13,attr_14,attr_1,attr_8,attr_5,attr_0
user_id,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.5,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,10.0,10.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
features_attr.to_csv(path_or_buf='features_attr.csv', index=False)